<a href="https://colab.research.google.com/github/vigneshnani1234/AI-ML-RAG-Other/blob/main/RAG_DEMO.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:


!pip install -qU \
    langchain-pinecone\
    langchain-text-splitters \
    langchain_groq\
    langchain \
    pinecone-notebooks\
    gradio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.6/59.6 MB 14.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 323.9/323.9 kB 23.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 82.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 131.1/131.1 kB 12.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.4/70.4 kB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.3/46.3 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 587.6/587.6 kB 44.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 240.0/240.0 kB 21.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.2/52.2 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.3/42.3 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 221.1/221.1 kB 18.1 MB/s eta 0:00:00


In [ ]:
!pip install chromadb langchain langchainhub langchain-community transformers sentence-transformers


In [ ]:
!pip install --upgrade pydantic


In [ ]:

import os
import time
from langchain_text_splitters import MarkdownHeaderTextSplitter
from langchain_pinecone import PineconeEmbeddings, PineconeVectorStore
from langchain.chains import RetrievalQA
from pinecone import Pinecone, ServerlessSpec
import gradio as gr
from langchain_groq import ChatGroq

In [ ]:


from google.colab import userdata
# available at platform.openai.com/api-keys
os.environ['GROQ_API_KEY'] = userdata.get('GROQ_API_KEY')

In [ ]:
from google.colab import userdata
os.environ['PINECONE_INDEX_API']=userdata.get('PINECONE_INDEX_API')

In [ ]:
from langchain_text_splitters import MarkdownHeaderTextSplitter


markdown_document = """
# Bella Vista Italian Restaurant Menu

## Appetizers (Antipasti)

1. Bruschetta al Pomodoro -
14
   Slices of creamy buffalo mozzarella layered with vine-ripened tomatoes and fresh basil leaves. Drizzled with extra virgin olive oil and 12-year-aged balsamic glaze.

3. Calamari Fritti - 16 Tender rings of calamari lightly dusted in seasoned flour and flash-fried to golden perfection. Served with our house-made spicy marinara sauce and lemon wedges. ## Main Courses (Primi Piatti) 4. Spaghetti alla Carbonara -
22
   Al dente spaghetti tossed in a creamy sauce made with farm-fresh eggs, pecorino Romano cheese, and crispy pancetta. Finished with freshly cracked black pepper and parsley.

5. Risotto ai Funghi Porcini -
24
   Layers of fresh pasta sheets, rich meat ragù made with ground beef and pork, creamy béchamel sauce, and melted mozzarella. Baked until golden and bubbly.

## Second Courses (Secondi Piatti)

7. Osso Buco alla Milanese -
30
   Whole Mediterranean sea bass stuffed with lemon and herbs, oven-roasted and filleted tableside. Served with roasted potatoes and grilled asparagus.

9. Pollo al Marsala - 28 Tender chicken breast sautéed with mushrooms in a rich Marsala wine sauce. Served with creamy polenta and seasonal vegetables. ## Desserts (Dolci) 10. Tiramisu -
12
    Layers of coffee-soaked ladyfingers and creamy mascarpone cheese, dusted with premium cocoa powder. Served with a shot of espresso on the side.

11. Panna Cotta -
9
    Two crispy pastry tubes filled with sweet ricotta cream, chocolate chips, and candied orange peel. Dusted with powdered sugar and served with pistachio gelato.

## Beverages (Bevande)

13. Espresso -
5
    Espresso topped with steamed milk and foam

15. Italian Sodas -
9 / (bottle) -
8 / (bottle) - $32
    Pinot Grigio - Crisp and refreshing with hints of pear and citrus

Our dishes are prepared with the finest ingredients, including imported Italian products and locally sourced organic produce. Gluten-free and vegetarian options are available upon request. Buon appetito!
"""

headers_to_split_on = [
    ("#", "Header 1"),
    ("##", "Header 2")
]

markdown_splitter = MarkdownHeaderTextSplitter(
    headers_to_split_on=headers_to_split_on, strip_headers=False
)
md_header_splits = markdown_splitter.split_text(markdown_document)

print(md_header_splits)

[Document(metadata={'Header 1': 'Bella Vista Italian Restaurant Menu', 'Header 2': 'Appetizers (Antipasti)'}, page_content='# Bella Vista Italian Restaurant Menu  \n## Appetizers (Antipasti)  \n1. Bruschetta al Pomodoro -\n14\nSlices of creamy buffalo mozzarella layered with vine-ripened tomatoes and fresh basil leaves. Drizzled with extra virgin olive oil and 12-year-aged balsamic glaze.  \n3. Calamari Fritti - 16 Tender rings of calamari lightly dusted in seasoned flour and flash-fried to golden perfection. Served with our house-made spicy marinara sauce and lemon wedges. ## Main Courses (Primi Piatti) 4. Spaghetti alla Carbonara -\n22\nAl dente spaghetti tossed in a creamy sauce made with farm-fresh eggs, pecorino Romano cheese, and crispy pancetta. Finished with freshly cracked black pepper and parsley.  \n5. Risotto ai Funghi Porcini -\n24\nLayers of fresh pasta sheets, rich meat ragù made with ground beef and pork, creamy béchamel sauce, and melted mozzarella. Baked until golden 

In [ ]:
import os
import time
from langchain_community.vectorstores import Chroma
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain.chains import RetrievalQA
from langchain_groq import ChatGroq


In [ ]:
embedding_model="intfloat/multilingual-e5-large"
embeddings=HuggingFaceEmbeddings(model_name=embedding_model)


/tmp/ipython-input-24-492528446.py:2: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings=HuggingFaceEmbeddings(model_name=embedding_model)


modules.json:   0%|          | 0.00/387 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/57.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/690 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/418 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/280 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/201 [00:00<?, ?B/s]

In [ ]:
chroma_persist_directory="./chroma_db"

In [ ]:
docsearch=Chroma.from_documents(
    documents=md_header_splits,
    embedding=embeddings,
    persist_directory=chroma_persist_directory
)

llm=ChatGroq(
    model_name='deepseek-r1-distill-llama-70b',
)

In [ ]:
qa=RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=docsearch.as_retriever()
)

In [ ]:
query1 = "What are the appetizer options available at Bella Vista Italian Restaurant?"

query2 = "What vegetarian main course options are available, and can you describe one in detail without missing its price?"


In [ ]:
query1_with_knowledge = qa.invoke(query1)
query1_without_knowledge = llm.invoke(query1)

print(query1_with_knowledge['result'])
print()

<think>
Alright, so the user is asking about the appetizer options at Bella Vista Italian Restaurant. I need to look through the provided menu context to find the appetizers. 

First, I'll scan the sections. The menu is divided into categories like Appetizers, Main Courses, Beverages, Second Courses, and Desserts. The user specifically wants the Antipasti, which is the Italian term for appetizers.

Looking at the Appetizers section, I see two items listed. The first one is Bruschetta al Pomodoro, which is priced at 14. It's described as slices of creamy buffalo mozzarella layered with vine-ripened tomatoes and fresh basil leaves, drizzled with olive oil and balsamic glaze. 

The second appetizer is Calamari Fritti, priced at 16. It's tender calamari rings dusted in seasoned flour and fried, served with spicy marinara sauce and lemon wedges.

I should present these options clearly, mentioning each item's name, price, and description. That should fully answer the user's question about th

In [ ]:


print(query1_without_knowledge.content)

<think>

</think>

As an AI, I don't have access to real-time information, including specific menus of individual restaurants like Bella Vista Italian Restaurant. For the most accurate and up-to-date appetizer options, I recommend checking their official website or contacting the restaurant directly.
